# Examples of running HypoXPy workflow to locate earthquakes
The original event catalog and phase picks are the output of the QuakeFlow with GAMMA associator.

Make sure you have hypoinverse, hypoDD, and ph2dt in your search PATH and executable.

In [ ]:
import os
import pandas as pd
import numpy as np
from hypoxpy.workflow import relocate
from hypoxpy import utils
import matplotlib.pyplot as plt

## 1. Setup key driving parameters

In [ ]:
#================= User-defined parameters =================#
# Again, make sure you have hypoinverse, hypoDD, and ph2dt in your search PATH and executable.
indir='input'
outdir='output'
if not os.path.exists(outdir):
    os.makedirs(outdir)
namebase='GAMMA'

"""
Data files
"""
station_file=os.path.join(indir,'GAMMA_station_list.json')
station_file_hypoinv=os.path.join(indir,'GAMMA_station_hypoinv.dat')
station_file_hypodd=os.path.join(indir,'GAMMA_station_hypodd.dat')

event_file=os.path.join(indir,'GAMMA_catalog.csv')
phase_file=os.path.join(indir,'GAMMA_picks.csv') #input original phase file.
phase_file_reformat_hypoinv=f'{indir}/{namebase}_phase_hypoinv.pha' #output reformatted phase file for hypoDD.
phase_file_reformat_hypodd=f'{indir}/{namebase}_phase_hypodd.pha' #output reformatted phase file for hypoDD.

#final catalogs
#file names for the final summary catalogs.
out_hypoinv_bad = '%s/%s_hypoinv_bad.csv'%(outdir,namebase)
out_hypoinv_good = '%s/%s_hypoinv_good.csv'%(outdir,namebase)
out_hypodd_final = '%s/%s_hypodd_catalog.csv'%(outdir,namebase)

evid_label='event_id' #column name for event ID in the input event and pick files.
mapping_evid=True #whether to remap event IDs to integers starting from 1 for HypoDD compatibility.
        #If False, original event IDs are used (may cause issues if IDs are not integers or not starting from 1).
        #THIS SHOULD BE TRUE, UNLESS YOU ARE SURE YOUR EVENT IDS ARE INTEGERS STARTING FROM 1.
if mapping_evid:
      evid_label_mapped=evid_label + "_mapped" #this will be used only if mapping_evid is True.
else:
      evid_label_mapped = evid_label

save_cleaned_data=True #whether to save cleaned event and pick data after removing invalid entries.
        #THIS SHOULD BE TRUE IF mapping_evid IS TRUE. IT IS SET AUTOMATICALLY IN THE FUNCTION IF mapping_evid IS TRUE.
cleaned_eventfile = os.path.splitext(event_file)[0] + "_cleaned.csv"
cleaned_pickfile = os.path.splitext(phase_file)[0] + "_cleaned.csv"

# combine network and station for HypoDD format.
combine_net_sta=True
cleanup=True #remove intermediate files from each relocation run if True.
qc_phase=False #QC phase when reformatting. Currently, only check if both P and S are available. Recommend to set to False.

"""
Control parameters for running HypoInverse
"""
pmodel=os.path.join(indir,'velo_p_eg.cre')
smodel=os.path.join(indir,'velo_s_eg.cre')

depth_try_list=np.arange(0,20.5,1.0) #list of depths to try for each event during grid search.

min_nsta=4 #minimum number of stations to relocate the earthquake. 4 is recommeneded as the minimum to get a reliable location.
template_par_hypoinv = os.path.join(indir,'template_hypoinv_vp-vs.txt')

"""
Control parameters for running HypoDD
"""
#change the template parameters for more controls on hypoinverse, hypodd, and ph2dt.

template_par_ph2dt = os.path.join(indir,'template_ph2dt_par.inp')
template_par_hypodd = os.path.join(indir,'template_hypodd_par.inp')


### Pack parameters in a config dictionary

In [ ]:
hypox_pars = {
    "paths": utils.pick_vars("indir", "outdir", "namebase",scope=locals()),

    "preprocess": utils.pick_vars("save_cleaned_data", "cleaned_eventfile", "cleaned_pickfile",
                                  "combine_net_sta", "cleanup", "qc_phase",scope=locals()),

    "event_id": utils.pick_vars("evid_label", "mapping_evid", "evid_label_mapped",scope=locals()),

    "files": {
        "stations": {"json": station_file, "hypoinv": station_file_hypoinv,"hypodd": station_file_hypodd},
        "events": event_file,
        "phases": {"raw": phase_file,"hypoinv": phase_file_reformat_hypoinv,"hypodd": phase_file_reformat_hypodd},
        "final_catalogs": {"hypoinv_bad": out_hypoinv_bad,"hypoinv_good": out_hypoinv_good,"hypodd_final": out_hypodd_final},
    },

    "hypoinverse": {"p_model": pmodel,"s_model": smodel,"depth_list": depth_try_list,"min_nsta": min_nsta,"hypoinv_template": template_par_hypoinv},

    "hypodd": {"dep_corr": dep_corr,"ph2dt_template": template_par_ph2dt,"hypodd_template": template_par_hypodd},
}


## 2. Call the main driver

In [ ]:
relocate(hypox_pars, input_type="gamma", skip_hypoinverse=False, skip_hypodd=False, 
         allow_skip_hypoinverse=False,verbose=True)

## 3. Read and plot the earthquakes after HypoDD locations
Plot earthquakes scaled by magnitude and colored by depth

In [ ]:
#read station information
stations = utils.stainfo_json2csv(station_file,informat='GAMMA')
# drop duplicates ignoring 'channel'
stations_unique = stations.drop(columns=['channel']).drop_duplicates().reset_index(drop=True)
print(stations_unique)

quakes_original=pd.read_csv(cleaned_eventfile)
quakes_hypoinv=pd.read_csv(out_hypoinv_good)
quakes_hypodd=pd.read_csv(out_hypodd_final)

In [ ]:
size_scale = 5
# define min/max depth for color scale (km)
depth_min = 0
depth_max = 80


fig, axes = plt.subplots(1, 2, figsize=(11, 4), sharey=True)
mean_lat = np.mean(
    np.concatenate([quakes_original.latitude.values,
                    quakes_hypoinv.latitude.values,
                    quakes_hypodd.latitude.values])
)

for ax in axes:
    ax.set_aspect(1 / np.cos(np.deg2rad(mean_lat)))

# ---------- Panel 1: GAMMA -> HypoInv ----------
ax = axes[0]
ax.scatter(quakes_original.longitude, quakes_original.latitude,
           size_scale * np.power(2, quakes_original.magnitude),
           facecolors='0.5', edgecolors='none', label='GAMMA (%d)'%(len(quakes_original)))
sc1 = ax.scatter(quakes_hypoinv.longitude, quakes_hypoinv.latitude,
                 size_scale * np.power(2, quakes_hypoinv.magnitude),
                 quakes_hypoinv.depth_km,edgecolors='k', cmap='viridis_r', 
                 vmin=depth_min, vmax=depth_max, label='HypoInv (%d)'%(len(quakes_hypoinv)))
cbar1 = fig.colorbar(sc1, ax=ax, label='Depth (km)')
cbar1.ax.invert_yaxis()

#plot stations
ax.scatter(stations_unique.longitude,stations_unique.latitude,
           marker='^', facecolors='none',edgecolors='r', s=50, label='Stations')
ax.set_title('GAMMA -> HypoInv')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend(loc='lower center',ncol=2)

# ---------- Panel 2: GAMMA -> HypoInv -> HypoDD ----------
ax = axes[1]
ax.scatter(quakes_original.longitude, quakes_original.latitude,
           size_scale * np.power(2, quakes_original.magnitude),
           facecolors='0.5', edgecolors='none', label='GAMMA')
ax.scatter(quakes_hypoinv.longitude, quakes_hypoinv.latitude,
           size_scale * np.power(2, quakes_hypoinv.magnitude),
           facecolors='pink', edgecolors='none', label='HypoInv')
sc2 = ax.scatter(quakes_hypodd.longitude, quakes_hypodd.latitude,
                 size_scale * np.power(2, quakes_hypodd.magnitude),
                 quakes_hypodd.depth_km,edgecolors='k', cmap='viridis_r', 
                 vmin=depth_min, vmax=depth_max, label='HypoDD (%d)'%(len(quakes_hypodd)))
cbar2 = fig.colorbar(sc2, ax=ax, label='Depth (km)')
cbar2.ax.invert_yaxis()
#plot stations
ax.scatter(stations_unique.longitude,stations_unique.latitude,
           marker='^', facecolors='none',edgecolors='r', s=50, label='Stations')
ax.set_title('GAMMA -> HypoInv -> HypoDD')
ax.set_xlabel('Longitude')
ax.legend(loc='lower center',ncol=2)

plt.suptitle('Earthquake Locations Refinement Workflow', fontsize=16, y=1.03)
plt.tight_layout()
plt.show()
